In [1]:
import sys,os

import pyglet
pyglet.options['shadow_window'] = False

import pyrender
import numpy as np
import trimesh
import igl
import math
MESH_PATH = 'meshes'
if not os.path.exists(MESH_PATH):
    print( 'cannot find /resources, please update MESH_PATH')
    exit(1)
else:
    print('found resources')

CURVE_PATH = 'meshes/curvatures'
if not os.path.exists(CURVE_PATH):
    print( 'cannot find /resources, please update CURVE_PATH')
    exit(1)
else:
    print('found resources')

DECOM_PATH = 'meshes/decomposition'
if not os.path.exists(DECOM_PATH):
    print( 'cannot find /resources, please update CURVE_PATH')
    exit(1)
else:
    print('found resources')

SMOOTH_PATH = 'meshes/smoothing'
if not os.path.exists(SMOOTH_PATH):
    print( 'cannot find /resources, please update CURVE_PATH')
    exit(1)
else:
    print('found resources')

import matplotlib
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix, diags, spdiags, lil_matrix
import scipy.sparse.linalg as ssl

# Loading Geo Tools
LIB_PATH = '../python_lib'
sys.path.append(LIB_PATH) 
from geo_tools import rd_helper


from scipy.spatial import cKDTree
from scipy.optimize import minimize

%load_ext autoreload
%autoreload 2

found resources
found resources
found resources
found resources


## Uniform Laplacian

In [2]:
#Task 1 - Uniform Laplace 

#build my triangle mesh
mesh1 = os.path.join(MESH_PATH, "bumpy-cube.obj")
assert os.path.exists(mesh1), 'cannot find' + mesh1

bump = trimesh.load(mesh1)

In [3]:
adjacency_list = {i: set() for i in range(len(bump.vertices))}
for face in bump.faces:
    for i in range(3):
        # Add edge connections to the adjacency list (undirected graph)
        adjacency_list[face[i]].update({face[(i+1) % 3], face[(i+2) % 3]})

#Computing the discrete Laplacian at each vertex
laplacian_vectors = np.zeros_like(bump.vertices)
for i, adjacent in adjacency_list.items():
    sum_vector = np.sum(bump.vertices[list(adjacent)], axis=0)
    laplacian_vectors[i] = sum_vector / len(adjacent) - bump.vertices[i]

# Computing the mean curvature H as half the norm of the Laplacian vector
mean_curvature_H = np.linalg.norm(laplacian_vectors, axis=1) / 2

In [4]:
# np.set_printoptions(threshold=np.inf) #show 'all' outputs
angle_deficit = np.zeros(len(bump.vertices))

# area associated with each vertex
face_areas = bump.area_faces
vertex_areas = np.zeros(len(bump.vertices))

# For each face, add one third of its area to each of its vertices
for i, face in enumerate(bump.faces):
    for vertex in face:
        vertex_areas[vertex] += face_areas[i] / 3

for face in bump.faces:
    for i in range(3):
        # Compute the lengths of the edges a, b, c
        a = np.linalg.norm(bump.vertices[face[i]] - bump.vertices[face[(i+1) % 3]])
        b = np.linalg.norm(bump.vertices[face[(i+1) % 3]] - bump.vertices[face[(i+2) % 3]])
        c = np.linalg.norm(bump.vertices[face[(i+2) % 3]] - bump.vertices[face[i]])

        # Compute the angles using the law of cosines
        angle_a = np.arccos((b**2 + c**2 - a**2) / (2 * b * c))
        angle_b = np.arccos((a**2 + c**2 - b**2) / (2 * a * c))
        angle_c = np.arccos((a**2 + b**2 - c**2) / (2 * a * b))

        # Update the angle deficit for each vertex of the face
        angle_deficit[face[i]] += angle_a
        angle_deficit[face[(i+1) % 3]] += angle_b
        angle_deficit[face[(i+2) % 3]] += angle_c

        # Area calculation with Heron's Formula
        s = (a + b + c) / 2
        area = np.sqrt(s * (s - a) * (s - b) * (s - c))

        # Update the area associated with each vertex of the face
        vertex_areas[face[i]] += area / 3
        vertex_areas[face[(i+1) % 3]] += area / 3
        vertex_areas[face[(i+2) % 3]] += area / 3

# Normalize the angle deficit by the total area around each vertex to get Gaussian curvature
gauss_curvature = (2 * np.pi - angle_deficit) / vertex_areas


In [5]:
def normalize_values(values):
    min_val = np.min(values)
    max_val = np.max(values)
    return (values - min_val) / (max_val - min_val)

mean_curvature_normalized = normalize_values(mean_curvature_H)
gaussian_curvature_normalized = normalize_values(gauss_curvature)

def curvature_to_color(curvature, colormap=plt.cm.rainbow):
    # Apply the colormap to the normalized curvature values to get RGBA colors
    colors = colormap(curvature)[:, :3]  # Discard the alpha channel
    # Convert colors from [0, 1] to [0, 255] and to integers
    colors = (colors * 255).astype(np.uint8)
    return colors

mean_curve_hot = curvature_to_color(mean_curvature_normalized)
gauss_curve_hot = curvature_to_color(gaussian_curvature_normalized)

In [6]:
def write_ply_with_colors(filename, vertices, faces, colors):
    with open(filename, 'w') as file:
        # PLY header with color properties
        file.write("ply\n")
        file.write("format ascii 1.0\n")
        file.write("element vertex {}\n".format(len(vertices)))
        file.write("property float x\n")
        file.write("property float y\n")
        file.write("property float z\n")
        file.write("property uchar red\n")
        file.write("property uchar green\n")
        file.write("property uchar blue\n")
        file.write("element face {}\n".format(len(faces)))
        file.write("property list uchar int vertex_index\n")
        file.write("end_header\n")
        
        # Write vertex data with colors
        for vertex, color in zip(vertices, colors):
            file.write("{} {} {} {} {} {}\n".format(vertex[0], vertex[1], vertex[2], color[0], color[1], color[2]))
        
        # Write face data
        for face in faces:
            file.write("3 {} {} {}\n".format(face[0], face[1], face[2]))  # +1 for 1-based indexing in PLY

In [7]:
write_ply_with_colors('mean_curvature.ply', bump.vertices, bump.faces, mean_curve_hot)
write_ply_with_colors('gaussian_curvature.ply', bump.vertices, bump.faces, gauss_curve_hot)


#get PLY file and visualise on meshlab


2. This task will have to be done by hand as it is a manual calculation question illustrating the way the first and second fundamental forms are computed



## Non-Uniform (Discrete Laplace-Beltrami)

In [8]:
#task 3 - Non uniform
#laplace beltrami calculation 

#load a new mesh not manipulated 
mesh2 = os.path.join(CURVE_PATH, "lilium_s.obj")
assert os.path.exists(mesh2), 'cannot find' + mesh2

lils = trimesh.load(mesh2)

mesh3 = os.path.join(CURVE_PATH, "plane.obj")
assert os.path.exists(mesh3), 'cannot find' + mesh3

plane = trimesh.load(mesh3)

In [9]:

def cotangent(v0, v1, v2):
    # Compute the cotangent of the angle at v0
    edge1 = v1 - v0
    edge2 = v2 - v0
    dot_product = np.dot(edge1, edge2)
    cross_product_norm = np.linalg.norm(np.cross(edge1, edge2))
    if cross_product_norm == 0:
        return 0
    return dot_product / cross_product_norm

def area_of_triangle(v0, v1, v2):
    # Compute the area of the triangle by Heron's formula
    edge1_length = np.linalg.norm(v1 - v0)
    edge2_length = np.linalg.norm(v2 - v0)
    edge3_length = np.linalg.norm(v2 - v1)
    s = (edge1_length + edge2_length + edge3_length) / 2
    return np.sqrt(s * (s - edge1_length) * (s - edge2_length) * (s - edge3_length))

def createCM(vertices, faces):
    num_vertices = vertices.shape[0]
    C = lil_matrix((num_vertices, num_vertices), dtype=np.float64)
    M = lil_matrix((num_vertices, num_vertices), dtype=np.float64)
    
    for f in faces:
        for i in range(3):
            j = (i + 1) % 3
            k = (i + 2) % 3
            
            vi = vertices[f[i]]
            vj = vertices[f[j]]
            vk = vertices[f[k]]
            
            # Cotangents
            cot_ij = cotangent(vi, vj, vk)
            cot_ik = cotangent(vi, vk, vj)
            
            # Update C matrix
            C[f[i], f[j]] += cot_ik
            C[f[i], f[k]] += cot_ij
            C[f[i], f[i]] -= cot_ik + cot_ij
            
            # Update M matrix with the area of triangle
            area = area_of_triangle(vi, vj, vk) / 3.0
            M[f[i], f[i]] += area
    
    C = csr_matrix(C) #c matrix 
    M = csr_matrix(diags([M.diagonal()], [0])) #M (non-inverse)
    return C, M

def discrete_LB(C, M):
    # Convert mass matrix to diagonal format
    # M = diags([M.diagonal()], [0])
    M_inv = diags([1.0 / M.diagonal()], [0])
    
    # Laplacian matrix
    L = M_inv.dot(C) #take inverse of M
    return L

# Function to compute mean curvature using the Laplace-Beltrami operator
def mean_curvature_LBO(vertices, faces):
    # Compute the Laplace-Beltrami operator
    C,M = createCM(vertices, faces)
    LBO = discrete_LB(C,M)
    # Compute the mean curvature vector for each vertex
    H = -0.5 *LBO @ vertices
    # Compute the mean curvature magnitude for each vertex
    H_magnitude = np.linalg.norm(H, axis=1) / 2
    return H_magnitude


In [10]:
#call the functions 
# get verticies and faces 
mc_lils = mean_curvature_LBO(lils.vertices, lils.faces)
mc_plane = mean_curvature_LBO(plane.vertices, plane.faces)

# Normalize the mean curvature for visualization
mc_lils_normalized = normalize_values(mc_lils)
mc_plane_normalized = normalize_values(mc_plane)

# Apply color mapping
mc_lils_colors = curvature_to_color(mc_lils_normalized)
mc_plane_colors = curvature_to_color(mc_plane_normalized)

# Write the OBJ files with colors (you will need to create this function based on your visualization tool's capability)
write_ply_with_colors('lils_mean_curvature_colored.ply', lils.vertices, lils.faces, mc_lils_colors)
write_ply_with_colors('plane_mean_curvature_colored.ply', plane.vertices, plane.faces, mc_plane_colors)

## Modal Analysis

In [11]:
# loading in armidillo 
mesh4 = os.path.join(DECOM_PATH, "armadillo.obj")
assert os.path.exists(mesh4), 'cannot find' + mesh4

armi = trimesh.load(mesh4)

In [12]:

def compute_eigen_decomposition(v, f, k):
    L = igl.cotmatrix(v, f) #this was used instead of the original CM function as the mesh wasn't converging properly
    _, M= createCM(v,f)
    #LB = discrete_LB(L,M) somehow it plots the normals making a spiky mesh, non-smooth
    
    
    eigenvalues, eigenvectors = ssl.eigsh(L, k+1, M, sigma=1e-6, which='LM')
    return eigenvalues[1:], eigenvectors[:, 1:]

def reconstruct_mesh(v, eigenvectors):
    v_reconstructed = v + eigenvectors.dot(eigenvectors.T.dot(v - v.mean(axis=0)))
    return v_reconstructed


In [13]:
#set up our laplacian system 
v = armi.vertices
f = armi.faces

k_values = [5, 15, 200]  # Replace 50 with your desired large number
for k in k_values:
    eigenvalues, eigenvectors = compute_eigen_decomposition(v, f, k)
    
    # Reconstruct the mesh
    v_reconstructed = reconstruct_mesh(v, eigenvectors)
    
    # Create the mesh
    mesh_reconstructed = trimesh.Trimesh(vertices=v_reconstructed, faces=f)
    
    # Save the reconstructed mesh
    filename = f'armadillo_reconstructed_k{k}.ply'
    mesh_reconstructed.export(filename)

## Explicit Laplacian mesh smoothing

In [89]:
#loading in cube mesh and cow to test then illustrate complexity
#cube
mesh5 = os.path.join(MESH_PATH, "cube.obj")
assert os.path.exists(mesh5), 'cannot find' + mesh5
cube = trimesh.load(mesh5)
#cow
mesh6 = os.path.join(MESH_PATH, "camel.obj")
assert os.path.exists(mesh6), 'cannot find' + mesh6
camel = trimesh.load(mesh6)


In [29]:
def explicit_smoothing(V, F, lambda_step, iterations):
    L = igl.cotmatrix(V, F)
    
    # Smoothing with explicit integration
    for i in range(iterations):
        # Clamping lambda to ensure stability
        #lambda_step = min(lambda_step, 0.1 / np.max(np.abs(L)))
        V = V + lambda_step * L.dot(V)
    
    return V

In [90]:
# Convert mesh to the format expected by igl
V = camel.vertices
F = camel.faces


lambda_step = 1e-5 # 
iterations = 100   # 
V_smoothed = explicit_smoothing(V, F, lambda_step, iterations)
# # Compute the cotangent Laplacian matrix
# L = igl.cotmatrix(V, F)
camel.vertices = V_smoothed
camel.vertex_normals = camel.vertex_normals

camel.export('smoothed_mesh.obj')

# Visualize the smoothed mesh
# camel.show()

'# https://github.com/mikedh/trimesh\nv -0.00116903 36.48409136 44.32564005\nv 1.33018454 36.76915522 43.79539749\nv 1.48361328 36.14007157 43.29369074\nv -1.48588518 36.14004021 43.29351659\nv -1.33248649 36.76915523 43.79527546\nv 4.26736096 -31.83202370 -29.61903834\nv 4.21908840 -32.38336545 -29.71168448\nv 4.67034134 -31.97140205 -29.78506663\nv 8.28364638 -32.37700872 -21.71947151\nv 8.83525903 -32.50869600 -21.40786439\nv 8.88414250 -32.89207730 -21.42763969\nv 8.66864175 -30.90595770 -24.95640264\nv 8.05312672 -32.12713525 -26.65869091\nv 6.71032427 -32.88674346 -20.54092751\nv 6.76153918 -32.50358880 -20.56262978\nv 6.92933641 -32.37382134 -21.17283547\nv 6.71329139 -32.88733709 -23.70671186\nv 6.72897012 -32.18144282 -23.67552723\nv 6.97406356 -32.22483079 -23.52466015\nv 7.35077115 -29.97703900 -26.63312930\nv 7.16637412 -32.88863445 -23.61615024\nv 4.89936686 -30.09547639 -28.08894218\nv 5.48315207 -31.31001636 -29.21095899\nv -6.02187020 -32.22483080 13.40781770\nv -5.7819

## Implicit Laplacian mesh smoothing

In [68]:
#load in meshes
mesh7 = os.path.join(SMOOTH_PATH, "plane_ns.obj")
assert os.path.exists(mesh7), 'cannot find' + mesh7
pl_ns = trimesh.load(mesh7)
#cow
mesh8 = os.path.join(SMOOTH_PATH, "fandisk_ns.obj")
assert os.path.exists(mesh8), 'cannot find' + mesh8
fan_ns = trimesh.load(mesh8)



In [44]:
from scipy.sparse.linalg import cg
#implement the paper 
def implicit_smoothing(V, F, lambda_val, delta_t, iterations):
    L = igl.cotmatrix(V, F)
    _,M = createCM(V,F)

    # Precompute the system matrix for implicit smoothing
    A = M - lambda_val * delta_t * L
    
    # The A matrix will be used in a system A x = b, where
    # we need to solve for x given b. For stability, and to ensure
    # that we do not invert M at every iteration, we use the fact that
    # M is symmetric positive definite and hence can be used in the cg method directly.
    
    for i in range(iterations):
        # Solve the system for each dimension independently
        for d in range(V.shape[1]):  # Typically, V.shape[1] is 3 for 3D coordinates
            b = M.dot(V[:, d])
            V[:, d], _ = cg(A, b, maxiter=1000, tol=1e-10)
    return V



In [69]:
lambda_val = 1e-7  # Smoothing factor
delta_t = 1.0     # Time step size, adjust based on your mesh and needs
iterations = 50   # Number of iterations

# fan disk printing 
# Apply implicit smoothing
smoothed_vertices = implicit_smoothing(fan_ns.vertices, fan_ns.faces, lambda_val, delta_t, iterations)

# Update the mesh with the new vertices
fan_ns.vertices = smoothed_vertices

# Visualize the smoothed mesh
fan_ns.export('smoothed_mesh_imp.obj')
# fan_ns.show()

smoothed_vertices_plane = implicit_smoothing(pl_ns.vertices, pl_ns.faces, lambda_val = 1e-6, delta_t = 1.0, iterations = 50)

# Update the mesh with the new vertices
pl_ns.vertices = smoothed_vertices_plane

# Visualize the smoothed mesh
pl_ns.export('smoothed_mesh_imp_plane.obj')

'# https://github.com/mikedh/trimesh\nv -0.06705981 0.10735478 0.01179166\nv -0.09951461 0.10704439 0.01791202\nv -0.13304073 0.10718003 0.02161466\nv -0.16732483 0.10667904 0.01884399\nv -0.20032912 0.10768634 0.01869171\nv -0.23370057 0.10715340 0.01752957\nv -0.06734175 0.13292166 0.01536230\nv -0.10010917 0.13303157 0.01951013\nv -0.13310142 0.13332846 0.02150941\nv -0.16729504 0.13353220 0.01844361\nv -0.20017476 0.13288930 0.02031974\nv -0.23343388 0.13264417 0.02082358\nv -0.06698968 0.16640990 0.01859797\nv -0.09997023 0.16639086 0.02125474\nv -0.13339685 0.16689445 0.02106194\nv -0.16615437 0.16581313 0.02165319\nv -0.19932988 0.16662652 0.01943280\nv -0.23302615 0.16603718 0.01744152\nv -0.06663549 0.20028933 0.01360863\nv -0.10196067 0.20084324 0.02307099\nv -0.13374624 0.19977818 0.01929984\nv -0.16655064 0.20025868 0.01694418\nv -0.20025896 0.20015144 0.01527392\nv -0.23334078 0.20080373 0.01707251\nv -0.06719698 0.23358551 0.01410117\nv -0.09981494 0.23317474 0.01799129\n

## Performance Evaluation 

In [87]:
mesh9 = os.path.join(MESH_PATH, "dragon.obj")
assert os.path.exists(mesh9), 'cannot find' + mesh9
dragon = trimesh.load(mesh9)

In [71]:
def add_noise(mesh, std_dev):
    noisy_mesh = mesh.copy()
    noise = np.random.normal(0, std_dev, mesh.vertices.shape)
    noisy_mesh.vertices += noise
    return noisy_mesh

# Function to evaluate the denoised mesh against the original
def evaluate_denoising(original_mesh, denoised_vertices):
    mse = np.mean((original_mesh.vertices.shape[0] - denoised_vertices.vertices.shape[0]) ** 2)
    return mse


In [88]:
# Test parameters
#instead of a loop keep as a single iteration 
noise_levels = [0.01, 0.05, 0.001, 0.005]  # Example standard deviations for noise applied
results = []

std_dev = 0.01 #changed basis the choice in noise level
# Add noise to the clean mesh
mesh_test = dragon.copy()
noisy_mesh = add_noise(mesh_test, std_dev)
print(f"Noise Level: {std_dev}, applied to mesh")
noisy_mesh.export(f"noisy_mesh_{std_dev}.obj")
# Perform Laplacian smoothing
denoised_mesh_verticies = implicit_smoothing(noisy_mesh.vertices, noisy_mesh.faces, lambda_val = 1e-3, delta_t = 1.0, iterations = 50)
denoised_mesh = trimesh.Trimesh(vertices=denoised_mesh_verticies, faces=mesh_test.faces)
denoised_mesh.export(f"denoised_mesh_{std_dev}.obj")
# Evaluate the results
mse = evaluate_denoising(dragon, denoised_mesh)

# Store the results for later analysis
results.append((std_dev, mse))

# Now you can print the results or plot them to see how the error changes with noise level
for std_dev, mse in results:
    print(f"Noise Level: {std_dev}, MSE: {mse}")

Noise Level: 0.01, applied to mesh
Noise Level: 0.01, MSE: 196.0
